In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Activation
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Set random seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

# 1. Load and prepare the data
df = pd.read_csv('GOOG.csv', parse_dates=['Date'], index_col='Date')
df.sort_index(inplace=True)

# 2. Calculate essential technical indicators
def add_technical_indicators(df):
    # Moving Averages
    df['SMA_20'] = df['Close'].rolling(window=20).mean()
    df['EMA_20'] = df['Close'].ewm(span=20, adjust=False).mean()

    # Momentum
    df['Momentum'] = df['Close'] - df['Close'].shift(5)

    # Volatility
    df['Volatility'] = df['Close'].rolling(window=20).std()

    # Drop NA values
    df.dropna(inplace=True)
    return df

df = add_technical_indicators(df)

# 3. Split into train (2004-2018) and test (2019-2020)
train = df.loc['2004-01-01':'2018-12-31']
test = df.loc['2019-01-01':'2020-12-31']

# 4. Scale the data
scaler = MinMaxScaler(feature_range=(0, 1))
feature_cols = ['Open', 'High', 'Low', 'Close', 'Volume',
                'SMA_20', 'EMA_20', 'Momentum', 'Volatility']

train_scaled = scaler.fit_transform(train[feature_cols])
test_scaled = scaler.transform(test[feature_cols])

# 5. Create time sequences (using 3 months of historical data)
look_back = 60
def create_sequences(data, look_back=60):
    X, y = [], []
    for i in range(look_back, len(data)):
        X.append(data[i-look_back:i, :])
        y.append(data[i, feature_cols.index('Close')])
    return np.array(X), np.array(y)

X_train, y_train = create_sequences(train_scaled, look_back)
X_test, y_test = create_sequences(test_scaled, look_back)

# 6. Build LSTM model with ReLU activation
model = Sequential([
    LSTM(128, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])),
    Dropout(0.3),
    LSTM(64, return_sequences=False),
    Dropout(0.3),
    Dense(32),
    Activation('relu'),  # Using ReLU activation here
    Dense(1)
])

model.compile(optimizer='adam', loss='mse')

# 7. Train the model
history = model.fit(
    X_train, y_train,
    epochs=100,
    batch_size=32,
    validation_data=(X_test, y_test),
    callbacks=[EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)],
    verbose=1
)

# 8. Make predictions
predictions = model.predict(X_test)

# Inverse transform predictions
def inverse_transform_predictions(predictions, scaler, feature_index):
    dummy_array = np.zeros((len(predictions), len(feature_cols)))
    dummy_array[:, feature_index] = predictions.flatten()
    return scaler.inverse_transform(dummy_array)[:, feature_index]

y_pred = inverse_transform_predictions(predictions, scaler, feature_cols.index('Close'))
y_actual = inverse_transform_predictions(y_test, scaler, feature_cols.index('Close'))

# 9. Calculate and display metrics
metrics = {
    'RMSE': np.sqrt(mean_squared_error(y_actual, y_pred)),
    'MAE': mean_absolute_error(y_actual, y_pred),
    'MAPE': np.mean(np.abs((y_actual - y_pred) / y_actual)) * 100,
    'R²': r2_score(y_actual, y_pred)
}

print("\nEvaluation Metrics (2019-2020):")
for metric, value in metrics.items():
    print(f"{metric}: {value:.4f}")

# 10. Enhanced visualization
plt.figure(figsize=(16, 8))
test_dates = test.index[look_back:]

# Plot actual and predicted prices
plt.plot(test_dates, y_actual, label='Actual Price', color='blue', linewidth=2)
plt.plot(test_dates, y_pred, label='Predicted Price', color='red', linestyle='--', linewidth=1.5)

# Highlight COVID-19 period for context
covid_start = pd.to_datetime('2020-03-01')
covid_end = pd.to_datetime('2020-06-01')
plt.axvspan(covid_start, covid_end, color='gray', alpha=0.2, label='COVID-19 Impact')

plt.title('Google Stock Price: Actual vs Predicted (2019-2020)\nUsing ReLU Activation', fontsize=14)
plt.xlabel('Date', fontsize=12)
plt.ylabel('Price ($)', fontsize=12)
plt.legend(fontsize=12)
plt.grid(True, linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

# Plot training history
plt.figure(figsize=(12, 6))
plt.plot(history.history['loss'], label='Training Loss', linewidth=2)
plt.plot(history.history['val_loss'], label='Validation Loss', linewidth=2)
plt.title('Model Training Progress (ReLU Activation)', fontsize=14)
plt.xlabel('Epoch', fontsize=12)
plt.ylabel('Loss', fontsize=12)
plt.legend(fontsize=12)
plt.grid(True, linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

Epoch 1/100


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


111/111 ━━━━━━━━━━━━━━━━━━━━ 17s 114ms/step - loss: 0.0210 - val_loss: 0.0148
Epoch 2/100
111/111 ━━━━━━━━━━━━━━━━━━━━ 12s 110ms/step - loss: 0.0018 - val_loss: 0.0188
Epoch 3/100
 26/111 ━━━━━━━━━━━━━━━━━━━━ 11s 139ms/step - loss: 0.0016